# Leonardo's Aerial Screw: A Physics Investigation

> *"If this instrument made with a screw be well made – that is to say, made of linen of which the pores are stopped up with starch and be turned swiftly, the said screw will make its spiral in the air and it will rise high."*  
> — Leonardo da Vinci, Codex Atlanticus, f. 896r

## Introduction
Leonardo da Vinci's design for the "Aerial Screw" (c. 1489) is often considered the ancestor of the modern helicopter. While Leonardo believed the device would rise by compressing the air (like a screw into wood), modern aerodynamics tells us it works by generating lift through momentum transfer—similar to a propeller.

In this notebook, we will:
1.  **Analyze** the physics of the aerial screw using Blade Element Momentum Theory (BEMT).
2.  **Simulate** the performance of the screw at different helix angles and rotational speeds.
3.  **Explore** the trade-offs between lift, power, and efficiency interactively.

---

In [ ]:
# Install the davinci-codex library
!pip install -q git+https://github.com/Shannon-Labs/davinci-codex.git

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display

# Set "Brutalist Academic" plotting style
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman', 'DejaVu Serif'],
    'axes.grid': True,
    'grid.alpha': 0.3,
    'axes.facecolor': 'white',
    'figure.facecolor': 'white',
    'text.color': 'black',
    'axes.labelcolor': 'black',
    'xtick.color': 'black',
    'ytick.color': 'black'
})


## The Physics Model: Blade Element Momentum Theory

To accurately model the aerial screw, we treat it as a helical rotor. We divide the blade into small radial elements and calculate the forces on each one.

### 1. Blade Element Forces
For a blade element at radius $r$, the local velocity $V_{local}$ is the vector sum of the rotational velocity $\Omega r$ and the induced velocity $v_i$.

The lift $dL$ and drag $dD$ on a blade element of width $dr$ and chord $c$ are:

$$ dL = \frac{1}{2} \rho V_{local}^2 c C_l dr $$
$$ dD = \frac{1}{2} \rho V_{local}^2 c C_d dr $$

where $\rho$ is the air density, and $C_l, C_d$ are the lift and drag coefficients.

### 2. Momentum Theory
The rotor generates thrust by accelerating air downwards. The induced velocity $v_i$ is related to the thrust $dT$ by:

$$ dT = 2 \rho (2\pi r dr) v_i \sqrt{(\Omega r)^2 + v_i^2} $$

We solve these equations iteratively to find the equilibrium state.

In [ ]:
# Physical Constants
RHO_AIR = 1.225  # kg/m^3
ROTOR_RADIUS = 4.0  # m
INNER_RADIUS = 3.2  # m
BLADE_CHORD = 0.8   # m

class HelicalBladeElement:
    def __init__(self, radius, chord, twist):
        self.radius = radius
        self.chord = chord
        self.twist = twist

    def compute_forces(self, rpm, induced_velocity):
        omega = 2.0 * np.pi * rpm / 60.0
        tangential_velocity = omega * self.radius
        local_velocity = np.sqrt(tangential_velocity**2 + induced_velocity**2)
        
        inflow_angle = np.arctan2(induced_velocity, tangential_velocity)
        angle_of_attack = self.twist - inflow_angle
        
        # Simplified lift/drag model for demonstration
        # In a full simulation, we use lookup tables or more complex polars
        cl = 2 * np.pi * angle_of_attack  # Thin airfoil theory
        cd = 0.015 + 0.05 * angle_of_attack**2  # Drag polar
        
        # Stall modeling
        if abs(angle_of_attack) > np.radians(15):
            cl = 0.8 * np.sin(2 * angle_of_attack)
            cd = 0.2 + 0.5 * np.sin(angle_of_attack)**2

        q = 0.5 * RHO_AIR * local_velocity**2
        area = self.chord  # per unit length
        
        lift = q * area * cl
        drag = q * area * cd
        
        thrust = lift * np.cos(inflow_angle) - drag * np.sin(inflow_angle)
        torque = (lift * np.sin(inflow_angle) + drag * np.cos(inflow_angle)) * self.radius
        
        return thrust, torque

def simulate_rotor(helix_angle_deg, rpm):
    helix_angle_rad = np.radians(helix_angle_deg)
    num_elements = 20
    dr = (ROTOR_RADIUS - INNER_RADIUS) / num_elements
    
    total_thrust = 0
    total_torque = 0
    
    # Integrate along the blade
    for i in range(num_elements):
        r = INNER_RADIUS + (i + 0.5) * dr
        # Twist varies to maintain constant helix angle
        twist = np.arctan(np.tan(helix_angle_rad) * r / ROTOR_RADIUS) 
        
        element = HelicalBladeElement(r, BLADE_CHORD, twist)
        
        # Simple momentum theory iteration
        v_induced = 1.0 # Initial guess
        for _ in range(5):
            dT, dQ = element.compute_forces(rpm, v_induced)
            # Momentum update
            if dT > 0:
                v_new = np.sqrt(dT * dr / (2 * RHO_AIR * 2 * np.pi * r * dr))
                v_induced = 0.5 * v_induced + 0.5 * v_new
        
        total_thrust += dT * dr
        total_torque += dQ * dr
        
    power = total_torque * (2 * np.pi * rpm / 60.0)
    return total_thrust, power

## Interactive Simulation

Use the sliders below to adjust the **Helix Angle** (the pitch of the screw) and the **RPM** (rotation speed).

Observe how these parameters affect:
- **Thrust**: The upward force generated (Lift).
- **Power**: The energy required to turn the screw.
- **Efficiency**: The ratio of Thrust to Power (N/W).

In [ ]:
def plot_performance(helix_angle, rpm):
    thrust, power = simulate_rotor(helix_angle, rpm)
    
    # Calculate efficiency (N/kW for readability)
    efficiency = thrust / (power / 1000) if power > 0 else 0
    
    # Create plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Bar chart for Thrust and Power
    bars = ax1.bar(['Thrust (N)', 'Power (W)'], [thrust, power], color=['black', 'gray'])
    ax1.set_title(f'Performance at {helix_angle}° / {rpm} RPM')
    ax1.set_ylim(0, max(thrust, power) * 1.2)
    ax1.grid(axis='y', linestyle='--', alpha=0.5)
    
    # Add value labels
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}',
                ha='center', va='bottom')

    # Parameter Space Visualization
    # Show where current point sits on the curve
    angles = np.linspace(15, 45, 30)
    thrusts = []
    powers = []
    for a in angles:
        t, p = simulate_rotor(a, rpm)
        thrusts.append(t)
        powers.append(p)
        
    ax2.plot(angles, thrusts, 'k-', label='Thrust (N)')
    ax2.plot(angles, powers, 'k--', label='Power (W)')
    ax2.axvline(x=helix_angle, color='red', linestyle=':', label='Current Angle')
    ax2.set_xlabel('Helix Angle (deg)')
    ax2.set_title(f'Parameter Sweep at {rpm} RPM')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    print(f"Efficiency: {efficiency:.2f} N/kW")
    if power > 75:
        print("⚠️ Power exceeds sustainable human output (75 W)!")
    else:
        print("✅ Feasible for human power.")

interact(plot_performance, 
         helix_angle=widgets.FloatSlider(min=15, max=45, step=1, value=30, description='Helix Angle'),
         rpm=widgets.IntSlider(min=10, max=200, step=10, value=60, description='RPM'));

## Conclusion

Through this simulation, we can see that Leonardo's design faces significant challenges. While the screw *can* generate lift, the power required to lift a human (approx. 800N thrust needed) is far beyond what a human can produce (approx. 75W sustainable).

However, the **efficiency** peaks at specific helix angles, showing that Leonardo's intuition about the importance of the screw's shape was well-founded, even if the materials and power sources of his time were insufficient.